<a href="https://colab.research.google.com/github/MrTrongDo/DataTecHub/blob/main/StableDiffusion_edu_vn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="http://stablediffusion.edu.vn/images/banners/banner1.jpg" alt="Banner" width="100%">

 ## **STABLEDIFFUSION.EDU.VN - STABLE DIFFUSION GOOGLE COLAB - SETUP AND RUN**


 [![Badge with Link](https://img.shields.io/badge/WEBSITE-STABLEDEFFUSION.EDU.VN-Color?style=for-the-badge&logo=firefox&logoColor=green)](https://stablediffusion.edu.vn) &nbsp; &nbsp; &nbsp;[![](https://img.shields.io/badge/Nhóm-Facebook%20Group-%230080FF?style=for-the-badge&logo=facebook&logoColor=0080ff)](https://www.facebook.com/groups/stablediffusion.edu.vn) &nbsp; &nbsp; [![](https://img.shields.io/badge/FACEBOOK-Stable%20Diffusion%20AI%20Việt%20Nam-%230080FF?style=for-the-badge&logo=facebook&logoColor=0080ff)](https://www.facebook.com/StablediffusionVietNam) &nbsp; &nbsp; [![Badge with Link](https://img.shields.io/badge/Youtube-Học_stable_diffusion-%23ff0000?style=for-the-badge&logo=youtube&logoColor=ff0000)](https://www.youtube.com/@StableDiffusionAIVietNam)







In [ ]:
#@title #⚙️ 1. Kết nối Google Drive - Chọn Checkpoint

import os
import ipywidgets as widgets

from google.colab import drive
drive.mount('/content/drive')
%cd /content
print("Các bạn hãy để file Check point vào thư mục /content/drive/MyDrive/StableDiffusion/Model/")

def get_files_in_folder(folder_path):
    files = []
    for filename in os.listdir(folder_path):
        full_path = os.path.join(folder_path, filename)
        if os.path.isfile(full_path):
            files.append(filename)
    return files

folder_path = '/content/drive/MyDrive/StableDiffusion/Model/'  # Thay đổi thành đường dẫn của thư mục bạn muốn lấy danh sách file

sample= !ls $folder_path
if sample == ["ls: cannot access '" + folder_path + "': No such file or directory"] :
  print("Khởi tạo thư mục lần đầu")
  os.makedirs(folder_path)

file_list = get_files_in_folder(folder_path)

if len(file_list) > 0:
  ckpt = "/content/drive/MyDrive/StableDiffusion/Model/{}".format(file_list[0])
else :
  #download 1 checkpoint mới về và đưa vào dropdownlist
  Model_Link = "https://civitai.com/api/download/models/109123"
  %cd $folder_path
  !wget {Model_Link} --content-disposition
  file_list = get_files_in_folder(folder_path)

# Tạo dropdown list
dropdown = widgets.Dropdown(
    options=file_list,
    description='Chọn Checkpoint để chạy:',
    value=file_list[0],  # Giá trị mặc định được chọn ban đầu
)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        selected_word = change['new']
        print("Bạn đã chọn Checkpoint:", selected_word)
        ckpt = "/content/drive/MyDrive/StableDiffusion/Model/{}".format(selected_word)

dropdown.observe(on_change)

# Hiển thị dropdown list
display(dropdown)

In [ ]:
#@title #🚀 2. RUN - Lấy Ngrok token hoặc không thì để trống
import os
%cd /content

#Bật FastAPI tăng tốc cho khởi động Stable Diffusion
!pip install fastapi==0.90

!apt -y install -qq aria2

# Giao diện
#@markdown <br> [![](https://img.shields.io/badge/Lấy%20token-Ngrok-red)](https://dashboard.ngrok.com/get-started/your-authtoken)
Ngroktoken = "" #@param {type:"string"}
ckpt = ""

mode = 'dark'
sLinkFolder = "/content/drive/MyDrive/StableDiffusion/wildcards/links.txt"

#if not os.path.isfile(sLinkFolder) :
  #Nếu chạy SD lần đầu thì tải file link từ website về
  #!aria2c --continue --console-log-level=error -c -x 16 -s 16 -k 1M https://stablediffusion.edu.vn/Files/Links.txt -d /content/drive/MyDrive/StableDiffusion/wildcards -o links.txt

!wget https://stablediffusion.edu.vn/Files/links.txt -qq

#with open(sLinkFolder) as file:

with open("/content/links.txt") as file:
  links = file.readlines()

url = links[0]
repo_name_start_index = url.rfind("/") + 1
repo_name = url[repo_name_start_index:].strip()

gitUrl = url
gitFolder = repo_name

!git clone $gitUrl
#Extension
%cd $gitFolder/extensions
for link in links:
  !git clone $link

control_net_folder_name = links[1][(links[1].rfind("/") + 1):].strip()
print(control_net_folder_name)

%cd ..

#Controlnet install Preprocessor

#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/150_16_swin_l_oneformer_coco_100ep.pth -d /content/$gitFolder/models/oneformer -o 150_16_swin_l_oneformer_coco_100ep.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/250_16_swin_l_oneformer_ade20k_160k.pth -d /content/$gitFolder/models/oneformer -o 250_16_swin_l_oneformer_ade20k_160k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/ZoeD_M12_N.pt -d /content/$gitFolder/models/zoedepth -o ZoeD_M12_N.pt
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/dpt_beit_large_512.pt -d /content/$gitFolder/models/midas -o dpt_beit_large_512.pt
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/dpt_hybrid-midas-501f0c75.pt -d /content/$gitFolder/models/midas -o dpt_hybrid-midas-501f0c75.pt
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/dpt_large-midas-2f21e586.pt -d /content/$gitFolder/models/midas -o dpt_large-midas-2f21e586.pt
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/facenet.pth -d /content/$gitFolder/models/openpose -o facenet.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/hand_pose_model.pth -d /content/$gitFolder/models/openpose -o hand_pose_model.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/body_pose_model.pth -d /content/$gitFolder/models/openpose -o body_pose_model.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth -d /content/$gitFolder/models/keypose -o faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth -d /content/$gitFolder/models/keypose -o hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/latest_net_G.pth -d /content/$gitFolder/models/leres -o latest_net_G.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/res101.pth -d /content/$gitFolder/models/leres -o res101.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/mlsd_large_512_fp32.pth -d /content/$gitFolder/models/mlsd -o mlsd_large_512_fp32.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/netG.pth -d /content/$gitFolder/models/lineart_anime -o netG.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/network-bsds500.pth -d /content/$gitFolder/models/hed -o network-bsds500.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/scannet.pt -d /content/$gitFolder/models/normal_bae -o scannet.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/sk_model.pth -d /content/$gitFolder/models/lineart -o sk_model.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/sk_model2.pth -d /content/$gitFolder/models/lineart -o sk_model2.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/table5_pidinet.pth -d /content/$gitFolder/models/pidinet -o table5_pidinet.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/upernet_global_small.pth -d /content/$gitFolder/models/uniformer -o upernet_global_small.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lokCX/4x-Ultrasharp/resolve/main/4x-UltraSharp.pth -d /content/$gitFolder/models/ESRGAN -o 4x-UltraSharp.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge.pth -d /content/$gitFolder/models/softedge -o softedge.pth
#ControlNet Model

#Check thư mục
if os.path.exists("/content/drive/MyDrive/StableDiffusion/Controlnet") == False:
  !mkdir /content/drive/MyDrive/StableDiffusion/Controlnet
  !mkdir /content/drive/MyDrive/StableDiffusion/Controlnet/Models

  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny_fp16.safetensors -d /content/drive/MyDrive/StableDiffusion/Controlnet/Models -o control_v11p_sd15_canny.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_depth_fp16.safetensors -d /content/drive/MyDrive/StableDiffusion/Controlnet/Models -o control_v11f1p_sd15_depth.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart_fp16.safetensors -d /content/drive/MyDrive/StableDiffusion/Controlnet/Models -o control_v11p_sd15_lineart.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors -d /content/drive/MyDrive/StableDiffusion/Controlnet/Models -o control_v11p_sd15_mlsd.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose_fp16.safetensors -d /content/drive/MyDrive/StableDiffusion/Controlnet/Models -o control_v11p_sd15_openpose.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble_fp16.safetensors -d /content/drive/MyDrive/StableDiffusion/Controlnet/Models -o control_v11p_sd15_scribble.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg_fp16.safetensors -d /content/drive/MyDrive/StableDiffusion/Controlnet/Models -o control_v11p_sd15_seg_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors -d /content/drive/MyDrive/StableDiffusion/Controlnet/Models -o control_v11p_sd15s2_lineart_anime.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11u_sd15_tile_fp16.safetensors -d /content/drive/MyDrive/StableDiffusion/Controlnet/Models -o control_v11u_sd15_tile.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11u_sd15_tile_fp16.yaml -d /content/drive/MyDrive/StableDiffusion/Controlnet/Models -o control_v11u_sd15_tile.yaml
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors -d /content/drive/MyDrive/StableDiffusion/Controlnet/Models -o control_v11p_sd15_inpaint_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/DionTimmer/controlnet_qrcode-control_v1p_sd15/resolve/main/control_v1p_sd15_qrcode.safetensors -d /content/drive/MyDrive/StableDiffusion/Controlnet/Models -o QRcode.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/DionTimmer/controlnet_qrcode-control_v1p_sd15/resolve/main/control_v1p_sd15_qrcode.yaml -d /content/drive/MyDrive/StableDiffusion/Controlnet/Models -o QRcode.yaml
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ioclab/ioc-controlnet/blob/main/models/control_v1p_sd15_brightness.safetensors -d /content/drive/MyDrive/StableDiffusion/Controlnet/Models -o control_v1p_sd15_brightness.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/monster-labs/control_v1p_sd15_qrcode_monster/resolve/main/control_v1p_sd15_qrcode_monster.safetensors -d /content/drive/MyDrive/StableDiffusion/Controlnet/Models -o control_v1p_sd15_qrcode_monster.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge.pth -d /content/drive/MyDrive/StableDiffusion/Controlnet/Models -o control_v11p_sd15_softedge.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/CiaraRowles/TemporalNet/resolve/main/diff_control_sd15_temporalnet_fp16.safetensors -d /content/drive/MyDrive/StableDiffusion/Controlnet/Models -o diff_control_sd15_temporalnet_fp16.safetensors
# VAE FILE

if os.path.exists("/content/drive/MyDrive/StableDiffusion/VAE") == False:
  !mkdir /content/drive/MyDrive/StableDiffusion/VAE
  #!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d /content/drive/MyDrive/StableDiffusion/VAE -o VAE84.vae.pt
  #!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-ema-original/resolve/main/vae-ft-ema-560000-ema-pruned.safetensors -d /content/drive/MyDrive/StableDiffusion/VAE -o VAE56.vae.pt


#Custom

!mkdir /content/drive/MyDrive/StableDiffusion
!mkdir /content/drive/MyDrive/StableDiffusion/Model
!mkdir /content/drive/MyDrive/StableDiffusion/Setting
!mkdir /content/drive/MyDrive/StableDiffusion/Lora
!mkdir /content/drive/MyDrive/StableDiffusion/Embeddings
!mkdir /content/drive/MyDrive/StableDiffusion/wildcards
!mkdir /content/drive/MyDrive/StableDiffusion/Hypernetworks
!mkdir /content/drive/MyDrive/StableDiffusion/Outputs
!mkdir /content/$gitFolder/outputs

!ln -s /content/drive/MyDrive/StableDiffusion/wildcards /content/$gitFolder/extensions/sd-dynamic-prompts/

!aria2c --auto-file-renaming=false --console-log-level=error -c -x 16 -s 16 -k 1M https://raw.githubusercontent.com/minhnhat0709/sd-colab-config/main/config.json -d /content/drive/MyDrive/StableDiffusion/Setting -o config.json

#@title # 2. CHẠY STABLE DIFFUSION
ckpt = "/content/drive/MyDrive/StableDiffusion/Model/{}".format(dropdown.value)

if Ngroktoken != "" :
  !python launch.py --api --share --listen --theme dark --xformers --enable-insecure-extension-access --ui-config-file /content/drive/MyDrive/StableDiffusion/Setting/ui-config.json --ui-settings-file /content/drive/MyDrive/StableDiffusion/Setting/config.json --ckpt $ckpt --ckpt-dir /content/drive/MyDrive/StableDiffusion/Model/ --lora-dir /content/drive/MyDrive/StableDiffusion/Lora/ --embeddings-dir /content/drive/MyDrive/StableDiffusion/Embeddings --styles-file /content/drive/MyDrive/StableDiffusion/Setting/styles.csv --hypernetwork-dir /content/drive/MyDrive/StableDiffusion/Hypernetworks --ngrok {Ngroktoken} --controlnet-dir /content/drive/MyDrive/StableDiffusion/Controlnet/Models --vae-dir /content/drive/MyDrive/StableDiffusion/VAE/ --no-half-vae
else :
  !python launch.py --api --share --listen --theme dark --xformers --enable-insecure-extension-access --ui-config-file /content/drive/MyDrive/StableDiffusion/Setting/ui-config.json --ui-settings-file /content/drive/MyDrive/StableDiffusion/Setting/config.json --ckpt $ckpt --ckpt-dir /content/drive/MyDrive/StableDiffusion/Model/ --lora-dir /content/drive/MyDrive/StableDiffusion/Lora/ --embeddings-dir /content/drive/MyDrive/StableDiffusion/Embeddings --styles-file /content/drive/MyDrive/StableDiffusion/Setting/styles.csv --hypernetwork-dir /content/drive/MyDrive/StableDiffusion/Hypernetworks --controlnet-dir /content/drive/MyDrive/StableDiffusion/Controlnet/Models --vae-dir /content/drive/MyDrive/StableDiffusion/VAE/ --no-half-vae